# Модификации градиентного спуска

**Batch Gradient Descent**. По-русски её называют пакетным градиентным спуском, или ванильным градиентным спуском (хотя англоязычную вариацию Vanilla Gradient Descent чаще не переводят). По сути, это и есть классический градиентный спуск


**Стохастический градиентный спуск**. Слово «стохастический» можно воспринимать как синоним слова «случайный». Где же при использовании градиентного спуска может возникнуть случайность? При выборе данных. При реализации стохастического спуска вычисляются градиенты не для всей выборки, а только для случайно выбранной единственной точки.

**Mini-batch Gradient Descent**. Также можно называть его мини-пакетным градиентным спуском. По сути, эта модификация сочетает в себе лучшее от классической реализации и стохастического варианта. На данный момент это наиболее популярная реализация градиентного спуска, которая используется в глубоком обучении (т. е. в обучении нейронных сетей). В ходе обучения модели с помощью мини-пакетного градиентного спуска обучающая выборка разбивается на пакеты (батчи), для которых рассчитывается ошибка модели и пересчитываются веса.



In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
df = sns.load_dataset('diamonds')

In [2]:
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)

In [3]:
# кодировка категориальных признаков
df = pd.get_dummies(df, drop_first=True)

In [4]:
# логарифмируем признаки
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])

In [5]:
X = df.drop(columns="price")
y = df["price"]

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
from sklearn.linear_model import SGDRegressor

sgdr=SGDRegressor(random_state=42)

In [9]:
from sklearn.model_selection import GridSearchCV

In [10]:
param_grid={
    "loss": ["squared_error", "epsilon_insensitive"],
    "penalty": ["elasticnet"],
    "alpha": np.logspace(-3, 3, 15),
    "l1_ratio": np.linspace(0, 1, 11),
    "max_iter": np.logspace(0, 3, 10).astype(int),
    "random_state": [42],
    "learning_rate": ["constant"],
    "eta0": np.logspace(-4, -1, 4)       
}

In [11]:
grid_search=GridSearchCV(
    estimator=sgdr,
    param_grid=param_grid,
    n_jobs = -1
)

In [12]:
grid_search.fit(X_train, y_train)


KeyboardInterrupt: 

In [ ]:
print(grid_search.best_params_)

{'alpha': 0.0026826957952797246, 'eta0': 0.001, 'l1_ratio': 0.0, 'learning_rate': 'constant', 'loss': 'epsilon_insensitive', 'max_iter': 21, 'penalty': 'elasticnet', 'random_state': 42}


In [ ]:
best_sgdr=grid_search.best_estimator_

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
best_sgdr.fit(X_train, y_train)
y_pred_test=best_sgdr.predict(X_test)
print (round(mean_squared_error(y_test, y_pred_test), 3))

0.043


In [ ]:
def f(x):
    return x**3-72*x-220

def pr(x):
    return 3*(x**2)-72

In [ ]:
x0=12
x_cur=x0
 
i=1
while True:
    x_new=x_cur-f(x_cur)/pr(x_cur)
    if i>10:
        break
    x_cur=x_new
    i+=1
    print (x_new)

10.21111111111111
9.756461564762278
9.727252176332618
9.72713442131889
9.727134419408875
9.727134419408875
9.727134419408875
9.727134419408875
9.727134419408875
9.727134419408875


In [ ]:
def func1(x):
    return 24*(x**2)-4*x
def func2(x):
    return 48*x-4

In [ ]:
initial_value = 42
iter_count = 0
x_curr = initial_value
epsilon = 0.0001
f = func1(x_curr)

while (abs(f) > epsilon):
    f = func1(x_curr)
    f_prime = func2(x_curr)
    x_curr = x_curr - (f)/(f_prime)
    iter_count += 1
    print(x_curr)

21.041749502982107
10.562707090133793
5.323351550447383
2.7040050774153417
1.3949941413301903
0.7418109325525483
0.41784523900811205
0.26096925221473555
0.19169814030401197
0.16955770984744145
0.1667151339969682
0.1666666807529666
0.16666666666666785


In [ ]:
def newtons_method(f, der, eps, init):
    iter_count = 0
    x_curr = init
    f = f(x_curr)
    while (abs(f) > eps):
        f = f(x_curr)
        f_der = der(x_curr)
        x_curr = x_curr - (f)/(f_prime)
        iter_count += 1
    return x_curr

In [ ]:
from scipy.optimize import newton
newton(func=func1,x0=50,fprime=func2, tol=0.0001)

5.0

# Схема Бройдена — Флетчера — Гольдфарба — Шанно (BFGS)

In [13]:
import numpy as np
from scipy.optimize import minimize

In [14]:
# функция оптимизации
def func(x):
    return x[0]**2.0 + x[1]**2.0

In [16]:
# градиент
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])

In [17]:
x_0 = [1.0, 1.0]

In [18]:
result = minimize(func, x_0, method='BFGS', jac=grad_func)

In [19]:
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации Optimization terminated successfully.
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


Можно повторить то же самое с вариацией  L-BFGS-B:

In [20]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 + x[1]**2.0
 
#  определяем градиент функции
def grad_func(x):
    return np.array([x[0] * 2, x[1] * 2])
 
# определяем начальную точку
x_0 = [1, 1]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)
# получаем результат
print('Статус оптимизации %s' % result['message'])
print('Количество оценок: %d' % result['nfev'])
solution = result['x']
evaluation = func(solution)
print('Решение: f(%s) = %.5f' % (solution, evaluation))

Статус оптимизации CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Количество оценок: 3
Решение: f([0. 0.]) = 0.00000


In [31]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 - x[0]* x[1]+ x[1]**2.0 + 9.0 *x[0] - 6.0 * x[1] + 20.0
 
#  определяем градиент функции
def grad_func(x):
    return np.array([x[0] * 2 - x[1] + 9, x[1] * 2 - x[0] - 6])
 
# определяем начальную точку
x_0 = [-400, 400]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='BFGS', jac=grad_func)

In [22]:
result['x']

array([-4.,  1.])

In [32]:
# определяем нашу функцию
def func(x):
    return x[0]**2.0 - 3* x[0] +45
 
#  определяем градиент функции
def grad_func(x):
    return 2 * x[0] -3
 
# определяем начальную точку
x_0 = 10
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='BFGS', jac=grad_func)

In [33]:
result['nfev']

5

In [28]:
func(result['x'])

42.75

In [36]:
def func(x):
    return x[0]**4 + 6* x[1]**2 + 10
 
#  определяем градиент функции
def grad_func(x):
    return np.array([4* x[0]**3, 12*x[1]])
 
# определяем начальную точку
x_0 = [100, 100]
# реализуем алгоритм L-BFGS-B
result = minimize(func, x_0, method='L-BFGS-B', jac=grad_func)

In [38]:
func(result['x'])

10.00000000827103

# Линейное программирование

Линейное программирование — это метод оптимизации для системы линейных ограничений и линейной целевой функции. Целевая функция определяет оптимизируемую величину, и цель линейного программирования состоит в том, чтобы найти значения переменных, которые максимизируют или минимизируют целевую функцию.

## SciPy (scipy.optimize.linprog);

In [39]:
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

Здесь нам необходимо вспомнить линейную алгебру, так как очень важно, чтобы векторы были в нужных нам размерностях, иначе мы не сможем использовать матричное умножение. Вектор A размера 6 мы превращаем в матрицу размера (1, 6) с помощью функции expand_dims(). Создаём все необходимые переменные:

In [42]:
c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

In [43]:
from scipy.optimize import linprog
linprog(c=c, A_ub=A, b_ub=b)

     con: array([], dtype=float64)
     fun: -52.50000000003075
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-2.24904539e-11])
  status: 0
 success: True
       x: array([6.18738537e-14, 1.05853307e-12, 1.21475944e-13, 7.50000000e+00,
       4.00246695e-13, 4.71394166e-13])